<a href="https://colab.research.google.com/github/poring3995/Midterm_exam/blob/main/Exam_Spark_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark
     

#pip install spark
     

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

spark  

In [22]:
#Process 1 :  set the spark schemaInfernece to True To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

#Process 2 :  Create the streaming_df to readStream from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "/content/news/") \
    .option("maxFilesPerTrigger", 1) \
    .load("/content/input-5999222002/")

In [14]:
# Process 3 : I would like to check the schema of the data, so I place a sample json file and change readStream to read (expirimental process, so the uploaded version will be set with readStream instead)
streaming_df.printSchema()
streaming_df.show(truncate=False)

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)

+---------------------------------------+--------------+----------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|authors                                |category      |date      |headline                                                                                |link                                                                                                             |short_descri

In [23]:
# Process 4 : list the data heading column into the stream
from pyspark.sql.functions import explode, col

exploded_df = streaming_df \
    .select("authors", "category", "date", "headline", "link", "short_description") \
    

In [24]:
# Process 5 : Flatten listed the data 
flattened_df = exploded_df \
    .selectExpr("authors", "category", "date", "headline", "link", "short_description") 
print(flattened_df)

DataFrame[authors: string, category: string, date: string, headline: string, link: string, short_description: string]


In [29]:
# Process 6 :Write the output to console sink to check the output
writing_df = flattened_df.writeStream \
    .format("json") \
    .option("path", "/content/output-5999222002/") \
    .option("checkpointLocation","checkpoint_dir") \
    .outputMode("append") \
    .start()
    
#Process 7 : I interrupt with manual method in order to run the next processq
writing_df.awaitTermination()

KeyboardInterrupt: ignored

In [30]:
# Process 8 : Check the data at the output location

out_df = spark.read.json("/content/output-5999222002")
out_df.show(truncate=False)

+---------------------------------------+--------------+----------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|authors                                |category      |date      |headline                                                                                |link                                                                                                             |short_description                                                                                                                                                                                       |
+---------------------------------------+--------------+